### Para comenzar con el scrapping, importamos las funciones que tenemos en el archivo 'procesos'

In [39]:
from procesos import *

In [2]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import re

### Establecemos el nombre de las columnas que tendrá el dataframe

In [3]:
columnas = ['Obra','Status','Ejecución física %','Ejecución financiera %','Estado','Municipio','Dirección','Personas afectadas','Ente responsable','Funcionario responsable','Empresa contratista','Valor total de la obra USD (millones)','Obra financiada por FONDEN','Monto comprometido USD (millones)','Fecha estimada de inicio','Fecha real de inicio','Fecha culminación estimada de la obra','Fecha de paralización']

### Llamamos a la función 'obtener_datos_obra' que realiza el proceso de web scraping (selenium), obtenemos los datos que queremos de la obra y lo convertimos en un dataframe

In [4]:
url = 'https://transparenciave.org/obrasinconclusas/obras_inconclusas/empresa-socialista-de-riego-planicie-de-maracaibo/'
riego_maracaibo = obtener_datos_obra(url)

In [6]:
df = pd.DataFrame([riego_maracaibo]) #convertimos los datos en un dataframe
df = df.drop(df.columns[[2, 3]], axis=1) #eliminamos las columnas 2 y 3 que no tienen información
df.columns = columnas

df

,Obra,Status,Ejecución física %,Ejecución financiera %,Estado,Municipio,Dirección,Personas afectadas,Ente responsable,Funcionario responsable,Empresa contratista,Valor total de la obra USD (millones),Obra financiada por FONDEN,Monto comprometido USD (millones),Fecha estimada de inicio,Fecha real de inicio,Fecha culminación estimada de la obra,Fecha de paralización
0,EMPRESA SOCIALISTA DE RIEGO PLANICIE DE MARACAIBO,Paralizada,50%,S/I,Estado Zulia,Jesús Enrique Lossada,"Sector El Diluvio, parroquia José Ramón Yépez.",Entre 100.001 y 1.000.000,Instituto Nacional de Desarrollo Rural (INDER)...,S/I,"Constructora Norberto Odebrecht, C.A.",USD 142.171.747,Si,"USD 58.784.086,53",2003,S/I,2015,S/I


### Vamos a realizar el mismo procedimiento para las demás obras que tenemos en nuestra página web que estamos scrapeando

In [7]:
url2 = 'https://transparenciave.org/obrasinconclusas/obras_inconclusas/planta-procesadora-de-sisal/'
planta_sisal = obtener_datos_obra(url2)

In [11]:
df2 = pd.DataFrame([planta_sisal])
df2.columns = columnas

In [12]:
url3 = 'https://transparenciave.org/obrasinconclusas/obras_inconclusas/proyecto-agrario-integral-jose-inacio-de-abreu-e-lima/'
proy_agrario = obtener_datos_obra(url3)

In [15]:
df3 = pd.DataFrame([proy_agrario])
df3.columns = columnas

In [16]:
url4 = 'https://transparenciave.org/obrasinconclusas/obras_inconclusas/proyecto-agrario-socialista-de-desarrollo-integral-valle-de-quibor/'
valle_quibor = obtener_datos_obra(url4)

In [19]:
df4 = pd.DataFrame([valle_quibor])
df4.columns = columnas

In [22]:
url5 = 'https://transparenciave.org/obrasinconclusas/obras_inconclusas/puerto-de-aguas-profundas-puerto-cabello-fase-i/'
aguas_profundas = obtener_datos_obra(url5)

In [25]:
df5 = pd.DataFrame([aguas_profundas])
df5.columns = columnas

In [26]:
url6 = 'https://transparenciave.org/obrasinconclusas/obras_inconclusas/zona-industrial-de-carora/'
zona_ind = obtener_datos_obra(url6)

In [30]:
df6 = pd.DataFrame([zona_ind])

df6['Obra financiada por FONDEN'] = 'S/I' #agregamos las columnas que hacen falta para mantener el formato del DF
df6['Monto comprometido USD (millones)'] = 'S/I'

#Reorganizamos el orden de las columnas
df6 = df6[[0,1,2,3,4,5,6,7,8,9,10,11,'Obra financiada por FONDEN','Monto comprometido USD (millones)',12,13,14,15]]
df6.columns = columnas

df6

,Obra,Status,Ejecución física %,Ejecución financiera %,Estado,Municipio,Dirección,Personas afectadas,Ente responsable,Funcionario responsable,Empresa contratista,Valor total de la obra USD (millones),Obra financiada por FONDEN,Monto comprometido USD (millones),Fecha estimada de inicio,Fecha real de inicio,Fecha culminación estimada de la obra,Fecha de paralización
0,ZONA INDUSTRIAL DE CARORA,Paralizada,55%,55%,Estado Lara,Torres,Carora,Entre 5.001 y 100.000,Compañía Anónima para el Desarrollo Industrial...,S/I,"Constructora 421, C.A.",USD 263.125,S/I,S/I,2003,S/I,S/I,S/I


### Procedemos a juntar todos los dataframe

In [71]:
# Concatena los DataFrames en uno solo
data_prod = pd.concat([df,df2,df3,df4,df5,df6],ignore_index=True)

### Para conocer las coordenadas de cada una de las obras, hacemos uso de Geopy

In [72]:
#Asignamos como dirección el Municipio y Estado de cada obra para conocer sus coordenadas
municipio_estado = data_prod['Municipio'] + ', ' + data_prod['Estado']
lista_municipio_estado = municipio_estado.tolist()

In [73]:
latitudes = obtener_latitudes(lista_municipio_estado) #obtenemos la latitud de la obra

In [74]:
longitudes = obtener_longitudes(lista_municipio_estado) #obtenemos la longitud de la obra

In [75]:
data_prod['Latitud'] = latitudes #agregamos una nueva columna con todos los valores de las latitudes
data_prod['Longitud'] = longitudes #agregamos una nueva columna con todos los valores de las longitudes

### Extraemos la parte numérica de la columna 'Valor total de la obra' para tener un dato de tipo entero

In [76]:
# Aplicar la función a la columna 'Valor total de la obra'
data_prod['Valor total de la obra USD (millones)'] = data_prod['Valor total de la obra USD (millones)'].apply(extract_numeric_value)

data_prod['Valor total de la obra USD (millones)'] = data_prod['Valor total de la obra USD (millones)'].str.replace('.', '', regex=True).fillna('0').astype(int)

#Dividimos para tener el total de millones en un número mas legible
data_prod['Valor total de la obra USD (millones)'] = (data_prod['Valor total de la obra USD (millones)'] / 1000000).astype(int)

### Limpiamos la columna 'Monto comprometido', extraemos la parte numérica, llenamos los valores nulos con '0', transformamos a un tipo de dato float

In [77]:
data_prod['Monto comprometido USD (millones)'] = data_prod['Monto comprometido USD (millones)'].apply(extract_numeric_value)
data_prod['Monto comprometido USD (millones)'] = data_prod['Monto comprometido USD (millones)'].fillna('0')

# Convertir la columna a cadena, eliminar los puntos y reemplazar las comas por puntos
data_prod['Monto comprometido USD (millones)'] = data_prod['Monto comprometido USD (millones)'].astype(str)
data_prod['Monto comprometido USD (millones)'] = data_prod['Monto comprometido USD (millones)'].str.replace('.', '').str.replace(',', '.')

# Convertir la columna a tipo float
data_prod['Monto comprometido USD (millones)'] = pd.to_numeric(data_prod['Monto comprometido USD (millones)'], errors='coerce')

#Dividimos para tener el total de millones en un número más legible
data_prod['Monto comprometido USD (millones)'] = (data_prod['Monto comprometido USD (millones)'] / 1000000).astype(int)

### Cambiamos el tipo de dato de la columna 'Fecha estimada de inicio' por datetime y extraemos el año

In [78]:
# Convertir la columna 'Fecha estimada de inicio' a datetime
data_prod['Fecha estimada de inicio'] = pd.to_datetime(data_prod['Fecha estimada de inicio'], errors='coerce')
data_prod['Fecha estimada de inicio'] = data_prod['Fecha estimada de inicio'].dt.year

### Transformamos las columnas 'Ejecución física' y 'Ejecución financiera' en valores de tipo float, para ello llenamos los valores 'S/I' con 0 y extraemos la parte numérica

In [79]:
data_prod['Ejecución física %'] = data_prod['Ejecución física %'].apply(extract_numeric_value)
data_prod['Ejecución física %'] = data_prod['Ejecución física %'].str.replace(',', '.').replace({None: '0'})

# Convertir la columna a tipo float
data_prod['Ejecución física %'] = data_prod['Ejecución física %'].astype(float)

In [80]:
data_prod['Ejecución financiera %'] = data_prod['Ejecución financiera %'].apply(extract_numeric_value)
data_prod['Ejecución financiera %'] = data_prod['Ejecución financiera %'].str.replace(',', '.').replace({None: '0'})

# Convertir la columna a tipo float
data_prod['Ejecución financiera %'] = data_prod['Ejecución financiera %'].astype(float)

### Añadimos una nueva columna para indicar cuál es el sector correspondiente a la obra

In [81]:
data_prod['Sector'] = 'Productivo'

In [82]:
#Reordenamos las columnas
nuevo_orden = ['Sector','Obra','Status','Ejecución física %','Ejecución financiera %','Estado','Municipio','Dirección','Personas afectadas','Ente responsable','Funcionario responsable','Empresa contratista','Valor total de la obra USD (millones)','Obra financiada por FONDEN','Monto comprometido USD (millones)','Fecha estimada de inicio','Fecha real de inicio','Fecha culminación estimada de la obra','Fecha de paralización', 'Latitud', 'Longitud']
data_prod = data_prod[nuevo_orden]

In [83]:
data_prod

,Sector,Obra,Status,Ejecución física %,Ejecución financiera %,Estado,Municipio,Dirección,Personas afectadas,Ente responsable,...,Empresa contratista,Valor total de la obra USD (millones),Obra financiada por FONDEN,Monto comprometido USD (millones),Fecha estimada de inicio,Fecha real de inicio,Fecha culminación estimada de la obra,Fecha de paralización,Latitud,Longitud
0,Productivo,EMPRESA SOCIALISTA DE RIEGO PLANICIE DE MARACAIBO,Paralizada,50.00,0.0,Estado Zulia,Jesús Enrique Lossada,"Sector El Diluvio, parroquia José Ramón Yépez.",Entre 100.001 y 1.000.000,Instituto Nacional de Desarrollo Rural (INDER)...,...,"Constructora Norberto Odebrecht, C.A.",142,Si,58,2003.0,S/I,2015,S/I,10.590710,-72.075725
1,Productivo,PLANTA PROCESADORA DE SISAL,Paralizada,10.00,0.0,Estado Lara,Urdaneta,San Miguel,Entre 5.001 y 100.000,Alcaldía del municipio Urdaneta / Gobernación ...,...,Venchisa,2,Si,76,1999.0,2002,S/I,S/I,10.569389,-69.585037
2,Productivo,PROYECTO AGRARIO INTEGRAL JOSÉ INACIO DE ABREU...,Paralizada,43.51,0.0,Estado Anzoátegui,Aragua,Carretera Nacional El Tigre-Pariaguan ramal 41...,Más de 1.000.001,Instituto Nacional de Desarrollo Rural (INDER),...,"Constructora Norberto Odebrecht, C. A.",2039,Si,568,NaN,S/I,2da fase 31-12-2018,S/I,9.325075,-64.864814
3,Productivo,PROYECTO AGRARIO SOCIALISTA DE DESARROLLO INTE...,Paralizada,60.00,0.0,Estado Lara,Jiménez,Quibor,Entre 100.001 y 1.000.000,S/I,...,Grupo Queiroz Galvão S.A.,269,Si,511,2009.0,S/I,2016,S/I,10.064215,-69.363809
4,Productivo,PUERTO DE AGUAS PROFUNDAS PUERTO CABELLO FASE I,Paralizada,28.00,0.0,Estado Carabobo,Puerto Cabello,Sector La Salina,Entre 5.001 y 100.000,Ministerio de Transporte Terrestre,...,China Harbour Engineering Company ltd (CHEC).,1100,Si,25,2014.0,S/I,2017,S/I,10.467793,-68.043668
5,Productivo,ZONA INDUSTRIAL DE CARORA,Paralizada,55.00,55.0,Estado Lara,Torres,Carora,Entre 5.001 y 100.000,Compañía Anónima para el Desarrollo Industrial...,...,"Constructora 421, C.A.",0,S/I,0,2003.0,S/I,S/I,S/I,10.189435,-69.893117


### Exportamos nuestro dataframe

In [84]:
data_prod.to_csv('obras_productivo.csv', index=False)